In [5]:
import sys
import csv

import web_scrapping

In [ ]:
###################
# opening csv file and put it's content into a list of urls
###################
csv_contents = []

with open("resources/furniture stores pages.csv", "r") as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        csv_contents.append(row[0])

csv_contents.remove("max(page)")

print(len(csv_contents))

In [ ]:
###################
# Fetch HTML content for each URL
###################

html_contents = []

for url in csv_contents:
    html_content = web_scrapping.fetch_html_content(url)
    if html_content:
        html_contents.append(html_content)

print(f"Fetched HTML content from {len(html_contents)} URLs")

In [ ]:
###################
# Save fatched HTML content to file for faster following processing
###################

with open("resources/html_content.csv", "w", newline="") as f:
    write = csv.writer(f)
    write.writerow(html_contents)

In [3]:
###################
# setting new limit for csv reading because the html content si bigger that the actual size can read
###################

max_int = sys.maxsize

while True:
    # decrease the maxInt value by factor 10
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(max_int)
        break
    except OverflowError:
        max_int = int(max_int/10)

In [4]:
###################
# load fatched HTML content to file for faster following processing
###################

with open("resources/html_content.csv", "r") as f:
    read = csv.reader(f)

    html_contents = list(read)[0]

In [ ]:
print(html_contents[0])

In [ ]:
###################
# preprocessing HTML content by removing unecessary tags
###################
import preprocessing_html

for i in range(len(html_contents)):
    html_contents[i] = preprocessing_html.preprocess_html(html_contents[i])
    print(f"Preprocessed HTML content for analysis {i}/{len(html_contents)}")

In [ ]:
print(html_contents[0])
print(max(html_contents[0], key=len))


In [2]:
import findspark
findspark.init()

In [4]:
findspark.__version__

'2.0.1'

In [24]:
from pyspark.sql import SparkSession
import sparknlp
spark = SparkSession.builder.appName("HelloWorld").getOrCreate()
sc = spark.sparkContext

nums = sc.parallelize([1,2,3,4])
print(nums.map(lambda x: x*x).collect())


print("Spark NLP version: ", sparknlp.version())
print("FindSpark version: ", findspark.__version__)
print("spark version: ", spark.version)

[1, 4, 9, 16]
Spark NLP version:  5.0.2
FindSpark version:  2.0.1
spark version:  3.4.1


In [25]:
from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
from sparknlp.training import CoNLL
training_data = spark.read.text("resources/training_data.txt")
training_data.show()

+--------------------+
|               value|
+--------------------+
|Factory Buys 32cm...|
|Beadlight Cirrus ...|
|Hamar Plant Stand...|
|Arrange Console T...|
|Artemis Oval Mirr...|
|Trim Sideboard 18...|
|Aster Door Mat - ...|
|Hamar Plant Stand...|
|Linear Wood table...|
|  Aqua Table PRODUCT|
|Taylor Dining Tab...|
|A- Joint Tabla PR...|
|A- Joint Round Ta...|
|Helborn Table PRO...|
|Wrongwoods Table ...|
|Cork Dining Table...|
|Crystal Brook 11 ...|
|Hampton's 11 Piec...|
|Eastport 11 Piece...|
|Republic 13 Piece...|
+--------------------+
only showing top 20 rows



In [29]:
word_embeddings = WordEmbeddingsModel.pretrained("glove_100d")
# from sparknlp.annotator import BertEmbeddings
bert = WordEmbeddingsModel.load("resources/pretrained_models/glove.6B.100d.txt") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("bert") \
    .setCaseSensitive(False) \
    .setPoolingLayer(0)  # default 0


TypeError: 'JavaPackage' object is not callable

In [27]:
# load NER model trained by deep learning approach and GloVe word embeddings
ner_dl = NerDLModel.pretrained("ner_dl")
# load NER model trained by deep learning approach and BERT word embeddings
ner_bert = NerDLModel.pretrained("ner_dl_bert")

ner_dl download started this may take some time.


TypeError: 'JavaPackage' object is not callable

In [17]:
training_data.printSchema()

root
 |-- value: string (nullable = true)



In [6]:

nerTagger = NerDLApproach() \
    .setInputCols(["sentence", "token", "word_embeddings"]) \
    .setLabelColumn("label") \
    .setOutputCol("ner") \
    .setMaxEpochs(10) \
    .setLr(0.001) \
    .setPo(0.005) \
    .setBatchSize(64)

ner_model = nerTagger.fit(training_data)

ner_model.write().overwrite().save('resources/ner_model')

ner_model = NerDLModel.load('path/to/ner_model')
text = "The new iPhone was released today."
result = ner_model.transform(spark.createDataFrame([(text,)]).toDF("text"))
entities = result.select("token.result", "ner.result").collect()

for entity in entities[0][1]:
    print(entity)

Py4JJavaError: An error occurred while calling o127.fit.
: java.lang.Exception: Could not find a column of type word_embeddings. Make sure your pipeline is correct.
	at com.johnsnowlabs.storage.HasStorageRef$.$anonfun$getStorageRefFromInput$2(HasStorageRef.scala:70)
	at scala.Option.getOrElse(Option.scala:189)
	at com.johnsnowlabs.storage.HasStorageRef$.getStorageRefFromInput(HasStorageRef.scala:69)
	at com.johnsnowlabs.nlp.annotators.ner.dl.NerDLApproach.train(NerDLApproach.scala:490)
	at com.johnsnowlabs.nlp.annotators.ner.dl.NerDLApproach.train(NerDLApproach.scala:180)
	at com.johnsnowlabs.nlp.AnnotatorApproach._fit(AnnotatorApproach.scala:69)
	at com.johnsnowlabs.nlp.AnnotatorApproach.fit(AnnotatorApproach.scala:75)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:77)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(Deleg